### Testing and Validation of a Network with Batfish

Network engineers routinely need to verify configuration parameters of various devices. In a multi-vendor network this is very hard to do, even with automation tools. However, with Batfish and the vendor-independent models that it produces, this task is extremely simple.

In this notebook, we will look at configuration validation scenarios that can be implemented with Batfish. More specifically, we will examine how the configuration of NTP servers can be validated. The same validation scenarios can be performed for other configuration properties of nodes (dns-servers, tacacs-servers, snmp-community, VRFs, etc.), for properties of interfaces (mtu, bandwidth, input-access-list, output-access-list, state, etc.), VRFs, BGP sessions, and more.

In [1]:
# Importing packages and loading questions
%run startup.py

### Initializing our Network and Snapshot

In [2]:
NETWORK_NAME = "example_network"
SNAPSHOT_NAME = "example_snapshot"
SNAPSHOT_PATH = "../test_rigs/example"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'{\n  "answerElements" : [\n    {\n      "class" : "org.batfish.datamodel.answers.InitInfoAnswerElement",\n      "parseStatus" : {\n        "as1border1" : "PASSED",\n        "as1border2" : "PASSED",\n        "as1core1" : "PASSED",\n        "as2border1" : "PASSED",\n        "as2border2" : "PASSED",\n        "as2core1" : "PASSED",\n        "as2core2" : "PASSED",\n        "as2dept1" : "PASSED",\n        "as2dist1" : "PASSED",\n        "as2dist2" : "PASSED",\n        "as3border1" : "PASSED",\n        "as3border2" : "PASSED",\n        "as3core1" : "PASSED",\n        "host1" : "PASSED",\n        "host2" : "PASSED",\n        "iptables/host1.iptables" : "PASSED",\n        "iptables/host2.iptables" : "PASSED"\n      }\n    }\n  ],\n  "status" : "SUCCESS",\n  "summary" : {\n    "numFailed" : 0,\n    "numPassed" : 0,\n    "numResults" : 0\n  }\n}\n'

The network snapshot that we initialized above is illustrated below. You can download/view devices' configuration files [here](https://github.com/batfish/batfish/tree/master/test_rigs/example).

![example-network](https://raw.githubusercontent.com/batfish/batfish/master/test_rigs/example/example-network.png)

### Validating _NTP Servers_ Configuration
Given a reference set of valid _NTP Servers_, there are many possible validation scenarios for the configuration of _NTP servers_:
* Every node has at least one _NTP server_ configured.
* Every node has at least one _NTP server_ configured from the reference set.
* Every node has the reference set of _NTP servers_ configured.
* Every node has its specific reference set of _NTP servers_ stored in a database, JSON file or YAML file. We can extract this into a Python dictionary and use that for validation.

In [3]:
ref_ntp_servers = set(['23.23.23.23'])

In this example, we will use a reference NTP server set of [23.23.23.23]. Further, we will focus the validation on the **border** routers.<br>(We will be using [Pandas APIs](https://pandas.pydata.org/pandas-docs/stable/) with `Pybatfish` for viewing results and for doing validations.)

**First, let's retrieve the configured NTP servers for all border routers.**

In [4]:
# Setting the property column name that we want to extract from the nodes
COL_NAME = "ntp-servers"

In [5]:
node_props = bfq.nodeProperties(nodeRegex=".*border.*", propertySpec=COL_NAME).answer().frame()
node_props

,node,ntp-servers
0,as1border1,[]
1,as1border2,"[18.18.18.18, 23.23.23.23]"
2,as2border1,"[18.18.18.18, 23.23.23.23]"
3,as2border2,[18.18.18.18]
4,as3border1,"[18.18.18.18, 23.23.23.23]"
5,as3border2,"[18.18.18.18, 23.23.23.23]"


#### Every node has at least one _NTP server_ configured
Now that we have the list of servers, let's check the scenario where every node has at least one NTP server configured. This is accomplished by finding nodes where the list of configured NTP servers is empty.

In [6]:
# Violators
ns_violators = node_props[node_props[COL_NAME].apply(lambda x: len(x) == 0)]
ns_violators

,node,ntp-servers
0,as1border1,[]


(If you want to know more about the **lambda** keyword, see [lambda expressions](https://docs.python.org/3/reference/expressions.html#lambda).)

#### Every node has at least one _NTP server_ configured from the reference set
_NTP servers_ on all nodes should contain at least one _NTP server_ from our reference. This is a more lenient version of the previous check which verifies that every node has at least one _NTP server_ from our reference set.

To get a table of violators, we can use the following command.<br>If a node does not have even one common _NTP server_ with our reference set, we will count it as a violator.

In [7]:
# Violators (Nodes which do not contain even a single NTP server from our reference set)
ns_violators = node_props[node_props[COL_NAME].apply(lambda x: len(ref_ntp_servers.intersection(set(x))) == 0)]
ns_violators

,node,ntp-servers
0,as1border1,[]
3,as2border2,[18.18.18.18]


So `as1border1` contains an empty set of NTP servers which clearly violates our assertion in this case.

#### Every node has the reference set of _NTP servers_ configured.
A common use case for validating _NTP servers_ generally involves checking that the set of _NTP servers_ on all relevant nodes is equal to a given set. Doing this using `Pybatfish`/`pandas` is pretty straightforward.

An example command for getting a table of violators would look like:

In [8]:
# Violators (Nodes whose set of NTP servers is not equal to our reference set)
ns_violators = node_props[node_props[COL_NAME].apply(lambda x: ref_ntp_servers != set(x))]
ns_violators

,node,ntp-servers
0,as1border1,[]
1,as1border2,"[18.18.18.18, 23.23.23.23]"
2,as2border1,"[18.18.18.18, 23.23.23.23]"
3,as2border2,[18.18.18.18]
4,as3border1,"[18.18.18.18, 23.23.23.23]"
5,as3border2,"[18.18.18.18, 23.23.23.23]"


As we can see, all border nodes violate this condition.

To look at which nodes actually have different _NTP servers_ compared to our reference set, we can find the extra _NTP Servers_ and missing _NTP Servers_ for each node. Commands to do that would look like below:

In [9]:
ns_extra = node_props[COL_NAME].map(lambda x: set(x) - ref_ntp_servers)
ns_missing = node_props[COL_NAME].map(lambda x: ref_ntp_servers - set(x))
# Let's join these columns up with the node columns for a better view
diff_df = pd.DataFrame({'node': node_props["node"], 'extra-{}'.format(COL_NAME): ns_extra,
                        'missing-{}'.format(COL_NAME): ns_missing})
diff_df

,node,extra-ntp-servers,missing-ntp-servers
0,as1border1,{},{23.23.23.23}
1,as1border2,{18.18.18.18},{}
2,as2border1,{18.18.18.18},{}
3,as2border2,{18.18.18.18},{23.23.23.23}
4,as3border1,{18.18.18.18},{}
5,as3border2,{18.18.18.18},{}


#### Every node has its specific reference set of _NTP servers_ stored in a database
Every node should match its reference set of _NTP Servers_ which may be stored in a database or somewhere else. This sort of check enables easy validation of configurations which may be non-uniform across nodes, where each node could have a unique set of _NTP servers_ defined.

We will assume data from the database is fetched in the following format, where node names are dictionary keys and specific properties are defined in a property-keyed dictionary per node:


In [10]:
# Mock reference-node-data, presumably taken from some database
database = {'as1border1': {'ntp-servers': ['23.23.23.23'], 'dns-servers': ['1.1.1.1']},
            'as1border2': {'ntp-servers': ['23.23.23.23'], 'dns-servers': ['1.1.1.1']},
            'as2border1': {'ntp-servers': ['18.18.18.18', '23.23.23.23'], 'dns-servers': ['2.2.2.2']},
            'as2border2': {'ntp-servers': ['18.18.18.18'], 'dns-servers': ['1.1.1.1']},
            'as3border1': {'ntp-servers': ['18.18.18.18', '23.23.23.23'], 'dns-servers': ['2.2.2.2']},
            'as3border2': {'ntp-servers': ['18.18.18.18', '23.23.23.23'], 'dns-servers': ['2.2.2.2']},
            }

Note that there is an extra property in this dictionary that we don't care about comparing right now: `dns-server` (we will just filter this out below, before comparing the dataframe from `Batfish` to the one we are generating from the database).

After a little tweaking, the database and `Batfish` dataframes can be compared to generate two sets of servers: missing (defined in the database but not in the configurations) and extra (defined in the configurations but not in the database).

In [11]:
# Transpose database data so each node has its own row
database_df = pd.DataFrame(data=database).transpose()

# Index on node for easier comparison
df_node_props = node_props.set_index('node')

# Select only columns present in node_props (get rid of the extra dns-servers column)
df_db_node_props = database_df[df_node_props.columns].copy()

# Convert server lists into sets to support arithmetic below
df_node_props[COL_NAME] = df_node_props[COL_NAME].apply(set)
df_db_node_props[COL_NAME] = df_db_node_props[COL_NAME].apply(set)

# Figure out what servers are in the configs but not the database and vice versa
missing_servers = (df_db_node_props - df_node_props).rename(columns={COL_NAME: 'missing-{}'.format(COL_NAME)})
extra_servers = (df_node_props - df_db_node_props).rename(columns={COL_NAME: 'extra-{}'.format(COL_NAME)})
result = pd.concat([missing_servers, extra_servers], axis=1, sort=False)
# removing the index name for clearer output
del result.index.name
result

,missing-ntp-servers,extra-ntp-servers
as1border1,{23.23.23.23},{}
as1border2,{},{18.18.18.18}
as2border1,{},{}
as2border2,{},{}
as3border1,{},{}
as3border2,{},{}


### Continue exploring with Batfish!

We showed you how to extract the database of configured _NTP servers_ for every node and how to test that the settings are correct for a variety of desired test configurations. The underlying principles can be applied to other network configurations, such as [interfaceProperties](https://github.com/batfish/batfish/blob/master/questions/experimental/interfaceProperties.json), [bgpProperties](https://github.com/batfish/batfish/blob/master/questions/experimental/bgpProperties.json), [ospfProperties](https://github.com/batfish/batfish/blob/master/questions/experimental/ospfProperties.json) etc.

For example `interfaceProperties` question can be used to fetch properties like interface MTU through a simple command like:

In [12]:
interface_mtu = bfq.interfaceProperties(nodeRegex=".*border.*", propertySpec="mtu").answer().frame()
interface_mtu

,interface,mtu
0,as1border1:Ethernet0/0,1500
1,as1border1:GigabitEthernet0/0,1500
2,as1border1:GigabitEthernet1/0,1500
3,as1border1:Loopback0,1500
4,as1border2:Ethernet0/0,1500
5,as1border2:GigabitEthernet0/0,1500
6,as1border2:GigabitEthernet1/0,1500
7,as1border2:GigabitEthernet2/0,1500
8,as1border2:Loopback0,1500
9,as2border1:Ethernet0/0,1500


### Get involved with the Batfish community ! 

Start interacting through [Slack](https://join.slack.com/t/batfish-org/shared_invite/enQtMzA0Nzg2OTAzNzQ1LTUxOTJlY2YyNTVlNGQ3MTJkOTIwZTU2YjY3YzRjZWFiYzE4ODE5ODZiNjA4NGI5NTJhZmU2ZTllOTMwZDhjMzA) or [Github](https://github.com/batfish/batfish) to know more. We would love to talk with you about Batfish or your Network !